In [1]:
import os 
import torch
import torchvision
import torch.nn
from torch.autograd import Variable
from torch.autograd.gradcheck import zero_gradients
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import math
import sys
sys.path.append('../')
from output_utils import *

os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES']='7' 

In [5]:
transform = transforms.Compose([ transforms.ToTensor(),])
trainset = torchvision.datasets.CIFAR10(root='../../datasets/cifar10', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=200, shuffle=False, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='../../datasets/cifar10', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=200, shuffle=False, num_workers=2)

num_step = 20
max_perturb = 8/255
step_size = max_perturb*2.5/num_step

from model import *
model =  WideResNet().cuda()
file_path = 'model_robust.pt'
model.load_state_dict(torch.load(file_path))
model.cuda();
model.eval();
output_train_benign, output_train_adversarial, output_test_benign, output_test_adversarial, train_label, test_label =\
classifier_performance(model, train_loader, test_loader, num_step, max_perturb, step_size, clip_min=0, clip_max=1.0)

results_dir = './output_results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
np.savez(os.path.join(results_dir, 'model_robust.npz'), 
         output_train_benign = output_train_benign, output_test_benign = output_test_benign,
         output_train_adversarial = output_train_adversarial, output_test_adversarial = output_test_adversarial,
         train_label = train_label, test_label = test_label)

Files already downloaded and verified
Files already downloaded and verified
Benign accuracy:  (0.99504, 0.8799)  Adversarial accuracy:  (0.76058, 0.465)
